# End to end multi-class dog breed classification
This notebook builds an end to end multi class image classifier using Tensorflow 2.0 and Tensorflow Hub.

## 1.problem
Identifying the breed of a dog given an image of a dog. When I am sitting in a cafe and I take a photo of a dog, I want to know what breed of dog it is.

## 2.Data
The data we are using is from kaggle's dog breed identification competition.
https://www.kaggle.com/competitions/dog-breed-identification

## 3.Evaluation
The evaluation is a file with prediction probabilities for each dog breed of each test image
https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## 4.Features
Some information about the data:
* we are dealing with images(unstructured data), so it is probably best we use deep learning/transfer learning
* There are 120 breeds of dogs (This means there are 120 different classes).
* There are around 10,000+ images in the training set(These images have labels)
* There are around 10,000+ images in the test set(These images have no labels), because we will want to predict them.
https://www.kaggle.com/competitions/dog-breed-identification/data

Note: To Unzip the data in colab, use `!unzip "source path" -d "destination path"`

### Get the workspace ready

In [4]:
import tensorflow as tf
print(tf.__version__)

2.6.0
